# Prediction

In [23]:
import numpy as np
import os
import tensorflow as tf
import re
import pandas as pd
print(tf.__version__)
assert(tf.__version__.startswith("2."))

from tensorflow.keras import layers, backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import backend
assert(tf.__version__.startswith("2."))
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras import Input
from tensorflow.keras import Model, layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
       
# Tensorboard
from tensorflow.python.keras.callbacks import TensorBoard
import datetime

# Helper libraries
# from w266_common import utils, vocabulary, tf_embed_viz

# From sklearn 
from sklearn.model_selection import train_test_split

import time


from nltk import tokenize
from joblib import load
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

2.4.0


In [10]:
!pwd

/Users/okcat/Documents/MIDS/text-formality-classifier


In [26]:
def read_file(path, file):
    sentences = []
    f = open(path+file, "r")
    for line in f:
        sentences.append(line)
        #sentences.append(re.findall(r"[\w']+|[.,!?;-]", line.split("\n")[0]))
    # print(f.read())
    f.close()
    return sentences

In [27]:
path1 = '../GYAFC_Corpus/Entertainment_Music/' 
path2 = '../GYAFC_Corpus/Family_Relationships/'

informal = read_file(path1, file='train/informal') + read_file(path2, file='train/informal')
formal = read_file(path1, file='train/formal') + read_file(path2, file='train/formal')

len(informal), len(formal)

(104562, 104562)

In [28]:
informal[:5]

['the movie The In-Laws not exactly a holiday movie but funny and good!\n',
 'that page did not give me viroses(i think)\n',
 'of corse i be wachin it evry day, my fav charachter is Inuasha\n',
 'runescape.com (my kids love it) & funbrain.com  (educational)\n',
 "Is he gay?He was on Late Night with Conan O'Brien and he seemed pretty gay\n"]

In [5]:
texts =["the movie The In-Laws not exactly a holiday movie but funny and good!",
 "that page did not give me viroses(i think)",
 'of corse i be wachin it evry day, my fav charachter is Inuasha',
 'runescape.com (my kids love it) & funbrain.com  (educational)',
 "Is he gay?He was on Late Night with Conan O'Brien and he seemed pretty gay"]

In [3]:
def print_pred(texts, pred):
    for txt, p in zip(texts, pred):
        print("formal: ", txt) if p == 0 else print("informal: ", txt)

## Naíve Bayes

In [15]:
NB = load("model/raw_nb.jbl")
pred = NB.predict(texts)
print_pred(texts, pred)

informal:  the movie The In-Laws not exactly a holiday movie but funny and good!
formal:  that page did not give me viroses(i think)
informal:  of corse i be wachin it evry day, my fav charachter is Inuasha
informal:  runescape.com (my kids love it) & funbrain.com  (educational)
informal:  Is he gay?He was on Late Night with Conan O'Brien and he seemed pretty gay


/Users/okcat/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.19.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/okcat/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/okcat/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.19.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/okcat/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.19.1 when using version 0.22.1. This might lea

In [49]:
n = len(inf_nb)
inf_nb = NB.predict(informal)
f_nb = NB.predict(formal)

print('Correct Informal: ', sum(inf_nb), ' words, ', 100 * sum(inf_nb)/n, '%')
print('Correct Formal: ', n - sum(f_nb), ' words, ', 100 * (1 - sum(f_nb)/n), '%')

Correct Informal:  81325  words,  77.77682140739465 %
Correct Formal:  77106  words,  73.74189476100304 %


## Logistic Regression

In [16]:
LG = load("model/raw_lg.jbl")
pred = LG.predict(texts)
print_pred(texts, pred)

informal:  the movie The In-Laws not exactly a holiday movie but funny and good!
formal:  that page did not give me viroses(i think)
informal:  of corse i be wachin it evry day, my fav charachter is Inuasha
informal:  runescape.com (my kids love it) & funbrain.com  (educational)
informal:  Is he gay?He was on Late Night with Conan O'Brien and he seemed pretty gay


/Users/okcat/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator HashingVectorizer from version 0.19.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/okcat/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/okcat/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.19.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [48]:
inf_nb = LG.predict(informal)
f_nb = LG.predict(formal)

print('Correct Informal: ', sum(inf_nb), ' words, ', 100 * sum(inf_nb)/n, '%')
print('Correct Formal: ', n - sum(f_nb), ' words, ', 100 * (1 - sum(f_nb)/n), '%')

Correct Informal:  79740  words,  76.26097435014633 %
Correct Formal:  81182  words,  77.64006044260822 %


## Desicion Tree

In [17]:
DT = load("model/raw_dt.jbl")
pred = DT.predict(texts)
print_pred(texts, pred)

informal:  the movie The In-Laws not exactly a holiday movie but funny and good!
informal:  that page did not give me viroses(i think)
informal:  of corse i be wachin it evry day, my fav charachter is Inuasha
informal:  runescape.com (my kids love it) & funbrain.com  (educational)
informal:  Is he gay?He was on Late Night with Conan O'Brien and he seemed pretty gay


/Users/okcat/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/okcat/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [47]:
inf_nb = DT.predict(informal)
f_nb = DT.predict(formal)

print('Correct Informal: ', sum(inf_nb), ' words, ', 100 * sum(inf_nb)/n, '%')
print('Correct Formal: ', n - sum(f_nb), ' words, ', 100 * (1 - sum(f_nb)/n), '%')

Correct Informal:  104562  words,  100.0 %
Correct Formal:  0  words,  0.0 %


## LSTM 

In [42]:
tokenizer = Tokenizer(num_words = 20000)
tokenizer.fit_on_texts(texts)
sequences_new = tokenizer.texts_to_sequences(texts)
new_texts = pad_sequences(sequences_new, maxlen = 1000) 

model = load_model("model/LSTM.h5") 
pred = np.argmax(model.predict(new_texts), axis=1)
print_pred(texts, pred)

formal:  the movie The In-Laws not exactly a holiday movie but funny and good!
formal:  that page did not give me viroses(i think)
formal:  of corse i be wachin it evry day, my fav charachter is Inuasha
informal:  runescape.com (my kids love it) & funbrain.com  (educational)
formal:  Is he gay?He was on Late Night with Conan O'Brien and he seemed pretty gay


In [43]:
tokenizer = Tokenizer(num_words = 20000)
tokenizer.fit_on_texts(informal)
sequences_new = tokenizer.texts_to_sequences(informal)
new_texts = pad_sequences(sequences_new, maxlen = 1000) 

inf = np.argmax(model.predict(new_texts), axis=1)


tokenizer = Tokenizer(num_words = 20000)
tokenizer.fit_on_texts(formal)
sequences_new = tokenizer.texts_to_sequences(formal)
new_texts = pad_sequences(sequences_new, maxlen = 1000) 

f = np.argmax(model.predict(new_texts), axis=1)

In [46]:
print('Correct Informal: ', sum(inf), ' words, ', 100 * sum(inf)/n, '%')
print('Correct Formal: ', n - sum(f), ' words, ', 100 * (1 - sum(f)/n), '%')

Correct Informal:  65148  words,  62.30561771963046 %
Correct Formal:  45702  words,  43.708039249440525 %
